## Download dataset

_Dataset details_: https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification

In [2]:
# Download the GTZAN dataset
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

# Unzip the downloaded dataset
!unzip -q gtzan-dataset-music-genre-classification.zip

100% 1.21G/1.21G [00:09<00:00, 104MB/s]
100% 1.21G/1.21G [00:09<00:00, 132MB/s]


## Load data to pandas dataframe

In [ ]:
import pandas as pd

# Load data to dataframe
data_path = '/content/Data/features_3_sec.csv'
data = pd.read_csv(data_path)

data.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


## Split the data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Splitting data into features and labels
X = data.drop(['filename', 'label'], axis=1)
y = data['label']

# Encoding labels and feature normalization
encoder = LabelEncoder()
y = encoder.fit_transform(y)
X = StandardScaler().fit_transform(X)

# Split data into training and validation parts
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((7992, 58), (1998, 58), (7992,), (1998,))

## CNN Random Forest Ensemble

### Data preparation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical

# Data preparation for CNN model
X_val_reshaped = np.array(X_val).reshape(X_val.shape[0], X_val.shape[1], 1)
y_val_encoded = encoder.fit_transform(y_val)
y_val_categorical = to_categorical(y_val_encoded)

y_train_encoded = encoder.fit_transform(y_train)
y_train_categorical = to_categorical(y_train_encoded)

### Creating the CNN feature extractor

In [ ]:
# Defining the CNN feature extractor
activation = 'relu'

cnn_feature_extractor = Sequential([
    Conv1D(filters=64, kernel_size=5, activation=activation, input_shape=(X_val_reshaped.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=5, activation=activation),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation=activation)
])

dl_layer = Dense(len(np.unique(y_val)), activation='softmax')(cnn_feature_extractor.output)
cnn_model = Model(inputs=cnn_feature_extractor.input, outputs=dl_layer)

# Compiling the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Tranining the CNN feature extractor

In [ ]:
# Training the model
cnn_model.fit(X_train, y_train_categorical, epochs=25, batch_size=32, validation_data=(X_val_reshaped, y_val_categorical), verbose=1)

Epoch 1/25
250/250 [==============================] - 1s 5ms/step - loss: 0.0856 - accuracy: 0.9740 - val_loss: 0.6721 - val_accuracy: 0.8579
Epoch 2/25
250/250 [==============================] - 1s 5ms/step - loss: 0.0474 - accuracy: 0.9840 - val_loss: 0.5994 - val_accuracy: 0.8784
Epoch 3/25
250/250 [==============================] - 1s 5ms/step - loss: 0.0183 - accuracy: 0.9947 - val_loss: 0.6158 - val_accuracy: 0.8934
Epoch 4/25
250/250 [==============================] - 1s 5ms/step - loss: 0.0204 - accuracy: 0.9935 - val_loss: 0.5345 - val_accuracy: 0.8954
Epoch 5/25
250/250 [==============================] - 1s 5ms/step - loss: 0.0229 - accuracy: 0.9936 - val_loss: 0.5879 - val_accuracy: 0.8884
Epoch 6/25
250/250 [==============================] - 1s 5ms/step - loss: 0.0195 - accuracy: 0.9944 - val_loss: 0.6530 - val_accuracy: 0.8844
Epoch 7/25
250/250 [==============================] - 1s 5ms/step - loss: 0.0284 - accuracy: 0.9911 - val_loss: 0.6978 - val_accuracy: 0.8769
Epoch 

In [ ]:
y_val_predict = np.argmax(cnn_model.predict(X_val_reshaped), axis=1)

# Evaluation
report = classification_report(y_val_encoded, y_val_predict, target_names=LabelEncoder().fit(y_val).classes_.astype(str))
print(report)

63/63 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.88      0.91      0.89       208
           1       0.92      0.95      0.93       203
           2       0.85      0.83      0.84       186
           3       0.88      0.88      0.88       199
           4       0.91      0.91      0.91       218
           5       0.86      0.91      0.88       192
           6       0.96      0.95      0.95       204
           7       0.92      0.93      0.93       180
           8       0.92      0.89      0.90       211
           9       0.86      0.80      0.83       197

    accuracy                           0.90      1998
   macro avg       0.90      0.90      0.90      1998
weighted avg       0.90      0.90      0.90      1998



## Ensemble with random forest

In [ ]:
# Estimating prediction
from sklearn.ensemble import RandomForestClassifier

X_train_reshaped = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
cnn_model_features = cnn_model.predict(X_train_reshaped)

# Create and train the Random Forest classifier on the extracted features
random_forest_model = RandomForestClassifier(n_estimators=10)
random_forest_model.fit(cnn_model_features, y_train)

250/250 [==============================] - 0s 2ms/step


RandomForestClassifier(n_estimators=10)

In [ ]:
val_features = cnn_model.predict(X_val_reshaped)
random_forest_predictions = random_forest_model.predict(val_features)

# Evaluation
report = classification_report(y_val_encoded, random_forest_predictions, target_names=LabelEncoder().fit(y_val).classes_.astype(str))
print(report)
random_forest_predictions

63/63 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       0.88      0.90      0.89       208
           1       0.90      0.95      0.93       203
           2       0.85      0.83      0.84       186
           3       0.89      0.88      0.88       199
           4       0.91      0.91      0.91       218
           5       0.87      0.91      0.89       192
           6       0.95      0.95      0.95       204
           7       0.92      0.93      0.93       180
           8       0.92      0.89      0.90       211
           9       0.85      0.81      0.83       197

    accuracy                           0.90      1998
   macro avg       0.89      0.89      0.89      1998
weighted avg       0.90      0.90      0.90      1998



array([4, 5, 0, ..., 4, 3, 8])

## Ensemble with KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

features = cnn_feature_extractor.predict(X_train)
val_features = cnn_feature_extractor.predict(X_val)

# Step 3: Train the KNN classifier
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(features, y_train)

63/63 [==============================] - 0s 2ms/step


KNeighborsClassifier(n_neighbors=3)

In [ ]:
val_features = cnn_feature_extractor.predict(X_val)
knn_predictions = knn_model.predict(val_features)

report = classification_report(y_val_encoded, knn_predictions, target_names=LabelEncoder().fit(y_val).classes_.astype(str))
print(report)

knn_predictions

63/63 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.88      0.93      0.90       208
           1       0.92      0.98      0.95       203
           2       0.84      0.85      0.84       186
           3       0.91      0.89      0.90       199
           4       0.94      0.94      0.94       218
           5       0.90      0.92      0.91       192
           6       0.97      0.98      0.97       204
           7       0.96      0.94      0.95       180
           8       0.95      0.92      0.94       211
           9       0.92      0.83      0.87       197

    accuracy                           0.92      1998
   macro avg       0.92      0.92      0.92      1998
weighted avg       0.92      0.92      0.92      1998



array([4, 5, 0, ..., 4, 3, 8])